In [82]:
import os
#from os import listdir
#from os.path import isfile, join
from random import random, sample
import argparse
from tqdm import tqdm
import numpy as np
from PIL import Image

from imutils import video

import cv2

parser = argparse.ArgumentParser()

# input, output
parser.add_argument("--input_dir", help="where to get input images")
parser.add_argument("--output_dir", help="where to put output images")
parser.add_argument("--max_num_images", type=int, help="maximum number of images to take (omit to use all)", default=None)
parser.add_argument("--shuffle", action="store_true", help="shuffle image")

#args = parser.parse_args()

#args.num_augment, args.w, args.h, args.frac, args.frac_vary, args.max_ang, args.max_stretch, args.centered
# no aug
# n=1, w2, h2, frac=1.0, frac_vary=0.0, max_ang=0, max_stretch=0, centered=True

#min_w, min_h = 512

allowable_actions = ['none', 'quantize', 'trace', 'hed', 'segment', 'simplify']


def cv2pil(img):
    if len(img.shape) == 2 or img.shape[2]==1:
        cv2_im = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    else:
        cv2_im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pil_im = Image.fromarray(cv2_im.astype('uint8'))
    return pil_im

def pil2cv(img):
    pil_image = img.convert('RGB') 
    cv2_image = np.array(pil_image) 
    cv2_image = cv2.cvtColor(cv2_image, cv2.COLOR_RGB2BGR)
    cv2_image = cv2_image[:, :, ::-1].copy()
    return cv2_image
    
def try_make_dir(new_dir):
    if not os.path.isdir(new_dir):
        os.mkdir(new_dir)


In [2]:


# cycle through input images
images = [f for f in listdir(input_dir) if isfile(join(input_dir, f)) ]
sort_order = sorted(range(min(num_images if num_images is not None else 1e8, len(images))))
if shuffle:
    sort_order = sorted(sample(range(len(images)), min(num_images if num_images is not None else 1e8, len(images))))
images = [images[i] for i in sort_order]


NameError: name 'listdir' is not defined

In [153]:


#input_src = '../datasets/trump_sub_short.mp4'
input_src = '../datasets/futurium/landscape_subset'
shuffle = False
max_num_images = 5


ext = os.path.splitext(input_src)[1]
is_movie = ext.lower() in ['.mp4','.mov','.avi']



def get_frame_indexes(max_num_images, num_images, shuffle):
    if shuffle:
        sort_order = sample(range(num_images), min(max_num_images if max_num_images is not None else 1e8, num_images))
    else:
        sort_order = sorted(range(min(max_num_images if max_num_images is not None else 1e8, num_images)))
    return sort_order


zzz = 'dfkjsdfkjs'


def crop_rot_resize(img, frac, w2, h2, ang, stretch, centered):
    if img.height < h2 or img.width < w2:
        img = upsample(img, w2, h2)
    
    if stretch != 0:
        v = random() < 0.5
        h = 1.0 if not v else (1.0 + stretch)
        w = 1.0 if v else (1.0 + stretch)
        img = img.resize((int(img.width * w), int(img.height * h)), resample=Image.BICUBIC)
        
    if ang > 0:
        img = img.rotate(ang, resample=Image.BICUBIC, expand=False)
   
    ar = float(w2 / h2)
    h1, w1 = img.height, img.width

    if float(w1) / h1 > ar:
        h1_crop = max(h2, h1 * frac)
        w1_crop = h1_crop * ar
    else:
        w1_crop = max(w2, w1 * frac)
        h1_crop = w1_crop / ar

    #xr, yr = 0.275 + 0.45*random(), 0.275 + 0.45*random()
    xr, yr = random(), random()
    x_crop, y_crop = (w1 - w1_crop - 1) * xr, (h1 - h1_crop - 1) * yr
    h1_crop, w1_crop, y_crop, x_crop = int(h1_crop), int(w1_crop), int(y_crop), int(x_crop)
    img_crop = img.crop((x_crop, y_crop, x_crop+w1_crop, y_crop+h1_crop))
    img_resize = img_crop.resize((w2, h2))
    
    return img_resize


def augmentation(img):#, args):
    #n, w2, h2, frac, frac_vary, max_ang, max_stretch, centered = args.num_augment, args.w, args.h, args.frac, args.frac_vary, args.max_ang, args.max_stretch, args.centered
    n, w2, h2, frac, frac_vary, max_ang, max_stretch, centered = 2, 256, 256, 0.9, 0.05, 0, 0, False
    imgs = []
    for n in range(n):
        ang = max_ang * (-1.0 + 2.0 * random())
        frac_amt = frac + frac_vary * (-1.0 + 2.0 * random())
        stretch = max_stretch * (-1.0 + 2.0 * random())
        newimg = crop_rot_resize(img, frac_amt, w2, h2, ang, stretch, centered)
        imgs.append(newimg)
    return imgs


def process_image(orig_img):
    actions = ['s']
    imgs0, imgs1 = augmentation(orig_img), []
    
    for img0 in imgs0:
        img = img0
        for a in actions:
            if a != 'dkfsdf':
                pass
        imgs1.append(img)

    return imgs0, imgs1
    
    
def main(): #args
    
    output_dir = 'test55'
    action = 'trace,simplify'
    save_ext = 'png'
    save_mode = 'split' # 'combined', 'split', 'output_only'
    pct_test = 0.2
    num_per = 3
    min_w, min_h = 512, 512
    w, h = 1024, 512
    
    # split, pct_test>0         train/train_A,train_B   test/test_A,test_B
    # combined, pct_test>0      train/                  test/
    # output_only, pct_test>0   train/                  test/

    # split, pct_test=0         train_A,train_B
    # combined, pct_test=0      .
    # output_only, pct_test=0   . 
    
    # get list of actions
    actions = action.split(',')
    if False in [a in allowable_actions for a in actions]:
        raise Exception('one of your actions does not exist')

    # setup output directories
    output_dir = output_dir #args.output_dir
    output_train_dir = os.path.join(output_dir, 'train')
    output_test_dir = os.path.join(output_dir, 'test')
    
    if pct_test > 0:
        if save_mode == 'split':
            output_trainA_dir = os.path.join(output_train_dir, 'train_A')
            output_testA_dir = os.path.join(output_test_dir, 'test_A')
            output_trainB_dir = os.path.join(output_train_dir, 'train_B')
            output_testB_dir = os.path.join(output_test_dir, 'test_B')
        else:
            output_trainA_dir = output_train_dir
            output_testA_dir = output_test_dir
            output_trainB_dir = output_train_dir
            output_testB_dir = output_test_dir
    elif save_mode == 'split':
        output_trainA_dir = join(output_dir, 'train_A')
        output_trainB_dir = join(output_dir, 'train_B')
    else:
        output_trainA_dir = output_dir
        output_trainB_dir = output_dir
        
    try_make_dir(output_dir)   
    
    try_make_dir(output_train_dir)
    try_make_dir(output_trainA_dir)
    try_make_dir(output_trainB_dir)

    if pct_test > 0:
        try_make_dir(output_test_dir)
        try_make_dir(output_testA_dir)
        try_make_dir(output_testB_dir)
    
    # initialize input stream
    if is_movie:
        cap = cv2.VideoCapture(input_src)
        fps = video.FPS().start()
        num_images = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        pct_frames = list(np.linspace(0, 1, num_images))
        all_frames = get_frame_indexes(max_num_images, num_images, shuffle)

    else:
        images = [f for f in os.listdir(input_src) if os.path.isfile(os.path.join(input_src, f)) ]
        num_images = len(images)
        all_frames = get_frame_indexes(max_num_images, num_images, shuffle)

    # training/test split
    training = [1] * len(all_frames) * num_per
    if pct_test > 0:
        n_test = int(len(all_frames) * num_per * pct_test)
        test_per = 1.0 / pct_test
        test_idx = [int(test_per * (i+1) - 1) for i in range(n_test)]
        for t in test_idx:
            training[t] = 0


    # iterate through each input
    for idx_frame in tqdm(all_frames):

        if is_movie:
            pct_frame = pct_frames[idx_frame]
            frame = int(pct_frame * num_images)
            cap.set(1, frame);
            ret, img = cap.read()
            frame_name = 'frame%06d' % frame
            img = cv2pil(img)
        else:
            img_path = images[idx_frame]
            frame_name = os.path.splitext(img_path)[0]
            img = Image.open(os.path.join(input_src, img_path)).convert("RGB")

        # skip images which are too small
        if img.width < min_w or img.height < min_h:
            continue

        imgs0, imgs1 = process_image(img)

        for i, (img0, img1) in enumerate(zip(imgs0, imgs1)):
            
            out_name = '%05d%s_%s.%s' % (idx_frame, '_%02d'%i if num_per>1 else '', frame_name, save_ext)
            is_train = training[num_per * idx_frame + i]
            
            if save_mode == 'combined':
                output_dir = output_trainA_dir if is_train else output_testA_dir
                img2 = Image.new('RGB', (w * 2, h))     
                img2.paste(img0.convert('RGB'), (0, 0))
                img2.paste(img1.convert('RGB'), (w, 0))
                img2.save(os.path.join(output_dir, out_name))
            
            else:
                outputA_dir = output_trainA_dir if is_train else output_testA_dir
                img0.convert('RGB').save(os.path.join(outputA_dir, out_name))
                if save_mode == 'split':
                    outputB_dir = output_trainB_dir if is_train else output_testB_dir
                    img1.convert('RGB').save(os.path.join(outputB_dir, out_name))
                



main()
    

100%|██████████| 5/5 [00:00<00:00, 14.10it/s]


In [58]:
num_frames

600.0

In [88]:
5 / 0.23

21.73913043478261

In [146]:
all_frames = list(range(4))
pct_test = 0.2

training = [1] * len(all_frames)
if pct_test > 0:
    n_test = int(len(all_frames) * pct_test)
    test_per = 1.0 / pct_test
    print(n_test)
    test_idx = [int(test_per * (i+1) - 1) for i in range(n_test)]
    print(test_idx)
    for t in test_idx:
        training[t] = 0


0
[]


In [93]:
print(all_frames)

[0, 1, 2, 3, 4]


In [110]:
test_idx

[0, 3, 6, 9, 12, 15, 18, 21]

In [111]:
training = [1] * len(all_frames)

In [118]:
training[test_idx[0]]

1

In [113]:
training[test_idx]

TypeError: list indices must be integers or slices, not list